In [1]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc
from functools import partial
import os.path

In [2]:
bd.projects

Brightway2 projects manager with 13 objects:
	StepByStep
	bw2_class_2020_example_database
	bw2_class_2020_intro
	car comparison paper
	check_electricity_EES_paper
	check_impact_electricity
	default
	ecoinvent 3.6
	global_power_mix
	parameters - manual creation
	premise_scenarios
	simapro-ecoinvent-import
	trial-import_from_SimaPro
Use `projects.report()` to get a report on all projects.

In [3]:
bd.projects.set_current("ecoinvent 3.6")

In [4]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.5
	ecoinvent 3.6

## import Land-system Chang method
 > Global

In [5]:

method_category = "LandSystemChange_Global"

In [6]:
si = bi.SimaProLCIACSVImporter(
    filepath=os.path.join(
        r"C:\Users\ViteksPC\Documents\00-ETH_projects\AESAmethods\from_simapro"
        f"\PBsLCIAv072_{method_category}.csv"
    )
)

Extracted 1 methods in 0.01 seconds


In [7]:
# si.data[0].keys()

In [8]:
for method in si.data:
    print(method["name"], method["unit"])

('PBs-LCIA (baseline)', 'Land-system change - Global') %


In [9]:
si.data[0]["exchanges"][0]#.keys()

{'amount': 1.56e-12,
 'CAS number': '',
 'categories': ('Raw', 'land'),
 'name': 'Transformation, from forest',
 'unit': 'm2'}

In [11]:
si.apply_strategies()

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: link_iterable_by_fields
Applying strategy: match_subcategories
Applied 10 strategies in 0.37 seconds


In [12]:
si.statistics()

1 methods
22 cfs
22 unlinked cfs


(1, 22, 22)

In [15]:
link_by_name_and_categories = partial(
    bi.strategies.link_iterable_by_fields,
    other=bd.Database(bd.config.biosphere),
    kind="biosphere",
    fields=("name", "categories"),
)

In [16]:
si.apply_strategy(link_by_name_and_categories)
si.statistics()

Applying strategy: link_iterable_by_fields
1 methods
22 cfs
4 unlinked cfs


(1, 22, 4)

In [13]:
# list(si.unlinked)

In [17]:
# from the 5 unlinked:
# 1 has amount 0, can be dropped - Carbon dioxide, in air
# 2 others "Carbon dioxide" and "Carbon monoxide" are not in biosphere3, BUT
# "Carbon dioxide, fossil" 
# and "Carbon monoxide, fossil" and "Carbon monoxide, non-fossil" (with same CFs)
# have been linked (see check_equivalent_linked)
[(exc["name"], exc["categories"]) for exc in list(si.unlinked)]

[('Transformation, from forest, natural', ('natural resource', 'land')),
 ('Transformation, from forest, secondary', ('natural resource', 'land')),
 ('Transformation, to forest, natural', ('natural resource', 'land')),
 ('Transformation, to forest, used', ('natural resource', 'land'))]

In [18]:
def see_unlinked(imported_methods):
    unlinked_exc_names = [
        exc_unlinked["name"] for exc_unlinked in list(imported_methods.unlinked)
    ]
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            if exc["name"] in unlinked_exc_names:
                print("\t",
                    exc["name"],
                    exc["categories"],
                    exc["amount"],
                )
        print("\n")
        # if exc["amount"] != 0

In [19]:
def check_equivalent_linked(imported_methods, list_names):
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            for x in list_names:
                if x in exc["name"]:
                    print("\t",
                        exc["name"],
                        exc["categories"],
                        exc["amount"],
                    )
        print("\n")

In [20]:
def remove_duplicates_if_any(imported_methods):
    for ix in range(len(imported_methods.data)):
        current_exc = len(imported_methods.data[ix]["exchanges"])
        wo_duplicates_exc = len(
            {tuple(o["input"]) for o in imported_methods.data[ix]["exchanges"]}
        )
        print(
            imported_methods.data[ix]["name"][1], ": ", current_exc, wo_duplicates_exc,
        )
        if current_exc != wo_duplicates_exc:
            unique_exchanges = {
                tuple(o["input"]): o for o in imported_methods.data[ix]["exchanges"]
            }
            imported_methods.data[ix]["exchanges"] = list(unique_exchanges.values())
            print("Duplicates removed.\n")
        else:
            print("No duplicates.\n")

In [21]:
see_unlinked(si)

Land-system change - Global
	 Transformation, from forest, natural ('natural resource', 'land') 1.56e-12
	 Transformation, from forest, secondary ('natural resource', 'land') 1.56e-12
	 Transformation, to forest, natural ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, used ('natural resource', 'land') -1.56e-12




In [29]:
b3 = bd.Database("biosphere3")
[
    flow
    for flow in b3
    if "transformation, to forest" in flow["name"].lower()
]

['Transformation, to forest, primary (non-use)' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, extensive' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, secondary (non-use)' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, unspecified' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, intensive' (square meter, None, ('natural resource', 'land'))]

In [30]:
check_equivalent_linked(si, ["Transformation, to forest"])

Land-system change - Global
	 Transformation, to forest, extensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, natural ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, primary (non-use) ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, secondary (non-use) ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, unspecified ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, used ('natural resource', 'land') -1.56e-12




In [31]:
si.drop_unlinked()
si.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
1 methods
18 cfs
0 unlinked cfs


(1, 18, 0)

In [32]:
# check if there are duplicates (both numbers should be equal)
remove_duplicates_if_any(si)

Land-system change - Global :  18 10
Duplicates removed.



In [33]:
si.statistics()

1 methods
10 cfs
0 unlinked cfs


(1, 10, 0)

In [34]:
si.write_excel(method_category)

Wrote matching file to:
C:\Users\ViteksPC\AppData\Local\pylca\Brightway3\ecoinvent-36.d2073cbbe9aff3f7cc76c4f9a36d6789\output\lcia-matching-LandSystemChange_Global.xlsx


In [35]:
PATH_TO_METHODS = os.path.join(
    r"C:\Users\ViteksPC\Documents\00-ETH_projects\AESAmethods\bw_methods"
    f"\\aesa_{method_category}.xlsx")
print(PATH_TO_METHODS)

C:\Users\ViteksPC\Documents\00-ETH_projects\AESAmethods\bw_methods\aesa_LandSystemChange_Global.xlsx


In [24]:
# Go to excel and apply manual corrections
# store corrected excel to PATH_TO_METHODS

In [56]:
# [method for method in bd.methods if "ReCiPe" in str(method)]

In [53]:
method_category

'ClimateChange_EnergyImbalance'

In [74]:
# TODO: complete categories
# TODO: complete descriptions

# name=
# unit=
# authors=
# doi=
# description=
# filename=

categories = {
    (
        ("AESA (PBs-LCIA)", "climate change", "atmospheric CO2 concentration"),
        "ppm",
        "Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
        "10.1016/j.ecolind.2017.12.065",
        "description",
        "aesa_ClimateChange_AtmosphericCO2Concentration.xlsx",
    ),
    (
        ("AESA (PBs-LCIA)", "climate change", "energy imbalance at top-of-atmosphere"),
        "Wm-2",
        "Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
        "10.1016/j.ecolind.2017.12.065",
        "description",
        "aesa_ClimateChange_EnergyImbalance.xlsx",
    ),
    (
        ("AESA (PBs-LCIA)", "ozone depletion", "stratospheric O3 concentration"),
        "Dobson unit",
        "Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
        "10.1016/j.ecolind.2017.12.065",
        "description",
        "aesa_OzoneDepletion_StratosphericO3Concentration.xlsx",
    ),
    (
        ("AESA (PBs-LCIA)", "ocean acidification", "carbonate ion concentration"),
        "omega aragonite",
        "Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
        "10.1016/j.ecolind.2017.12.065",
        "description",
        "aesa_OceanAcidification_CarbontateIonConcentration.xlsx",
    ),
    (
        ("AESA (PBs-LCIA)", "biogeochemical flows", "nitrogen"),
        "Tg N",
        "Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
        "10.1016/j.ecolind.2017.12.065",
        "industrial and intentional biological fixation of N",
        "aesa_BiogeochemicalFlow_N.xlsx",
    ),
    (
        ("AESA (PBs-LCIA)", "biogeochemical flow", "phosphorus"),
        "Tg P",
        "Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
        "10.1016/j.ecolind.2017.12.065",
        "P flow from freshwater systems into the ocean",
        "aesa_BiogeochemicalFlow_P.xlsx",
    ),
}

In [59]:
new = bi.ExcelLCIAImporter(
    PATH_TO_METHODS,
    # name=("AESA (PBs-LCIA)", "climate change", "energy imbalance at top-of-atmosphere"),
    name=("AESA (PBs-LCIA)", "climate change", "atmospheric CO2 concentration"),
    description="here will go the description",
    # unit="Wm-2",
    unit="ppm",
    authors="Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
    doi="10.1016/j.ecolind.2017.12.065",
)

In [60]:
new.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.16 seconds


In [61]:
# excel can have empty lines
def drop_empty_lines(data):
    for method in data:
        method['exchanges'] = [obj for obj in method['exchanges'] if obj['name']]
    return data 

In [62]:
new.apply_strategy(drop_empty_lines)

Applying strategy: drop_empty_lines


In [63]:
# new.data

In [64]:
new.statistics()

1 methods
42 cfs
0 unlinked cfs


(1, 42, 0)

In [65]:
assert len(list(new.unlinked)) == 0

In [296]:
new.write_methods(overwrite=True, verbose=True)

Wrote 1 LCIA methods with 42 characterization factors


In [275]:
[met for met in bd.methods if "concentr" in str(met).lower()]

[('PBs-LCIA (baseline)', 'Climate change - CO2 concentration'),
 ('AESA', 'trial-CC_CO2concentration')]

In [277]:
l_new = bd.Method(('AESA', 'trial-CC_CO2concentration')).load()

In [278]:
l_or = bd.Method(('PBs-LCIA (baseline)', 'Climate change - CO2 concentration')).load()

In [279]:
len(l_new)

42

In [280]:
len(l_or)

42

In [285]:
[i for i in l_or if i not in l_new]

[]